In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import ctypes
from pycuda.compiler import SourceModule
import time

N = 50  # Adjust as needed

# Define the Row structure
class Row(ctypes.Structure):
    _fields_ = [("col1", ctypes.c_int), ("col2", ctypes.c_float)]

# Create some example row data
rows_data = [
    
]




In [2]:
def read_csv_to_list_rating(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r',encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        for line in file:
            values = line.strip().split(',')
            row = [int(values[0]), int(values[1]), float(values[2]), int(values[3])]
            matrix.append(row)
        return header,matrix
def read_csv_to_list_movies(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r',encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        for line in file:
            values = line.strip().split(',')
            row = [int(values[0]), str(values[1]), str(values[2])]
            matrix.append(row)
        return header,matrix
    

headerRating,matrixrating = read_csv_to_list_rating('ratings.csv')
headerMovies,matrixMovies = read_csv_to_list_movies('movies.csv')



totalRaters = int(matrixrating[-1][0]) #total rows
totalMovies = int(matrixMovies[-1][0])


d={}
#row 0  is userID

for row in matrixrating:
    if row[0] not in d:
        d[row[0]]=[]
    else:
        d[row[0]].append(list(row[1:4]))

for key, val in d.items():
    d[key] = sorted(val, key=lambda x: x[2])


for key,val in d.items():
    d[key] = d[key][::-1]


newD = {}

for key,val in d.items():
    newD[key] = d[key][:100]


totalUsers = len(newD)

# rows = np.array([(1, 5.0), (2, 6.0), (3, 7.0), (4, 8.0)], dtype=[('col1', np.int32), ('col2', np.float32)])


rows_data=[]
for i in range(1,(len(newD)+1)):
    row = newD[i]
    for j in range(100):
        if j<len(row):
            rows_data.append(Row(row[j][0],row[j][1]))
        else:
            rows_data.append(Row(0,0.0))
            

In [3]:
# Define the parallel matrix multiplication kernel
kernel_code = """
struct Row {
    int col1;
    float col2;
};

__global__ void matmul(float *A, float *B, int N, int raters, int movies,Row *rows) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int found=0;
    // Access Row data and print it
    int workingID;
    float workingRating;
    if (row < raters && col < movies) {
        for(int i=0;i<100;i++){
            int col1_value = rows[(row-1)*100+i].col1;
            float col2_value = rows[(row-1)*100+i].col2;
            if (col1_value == col){
                found=1;
                workingID=col1_value;
                workingRating=col2_value;
                break;
            }
        }
    }
    if(found){
        printf("FOUND rater-%d movieID-%d workingID-%d workingRating-%f\\n",row,col,workingID,workingRating);
    }
    if (row < raters && col < movies && found==1) {
        float value = 0.0;
        for (int k = 0; k < N; k++) {
            value += A[row * N + k] * B[k * movies + col];
        }
        //printf("%f\\n",value);
    }
    
}
"""


mod = SourceModule(kernel_code)



C:\Users\arshs\AppData\Local\Temp\ipykernel_15556\428213739.py:42: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule(kernel_code)


In [4]:

# matrixAMovies = np.zeros((totalMovies, N), dtype=np.float32)

# matrixBUsers = np.zeros((N, totalRaters), dtype=np.float32)

matrixBMovies = np.zeros((totalMovies, N), dtype=np.float32)

matrixAUsers = np.zeros((N, totalRaters), dtype=np.float32)

# Fill matrices with random values in the range -0.01 to 0.01
matrixAUsers = np.random.uniform(-0.01, 0.01, size=(totalRaters, N)).astype(np.float32)
matrixBMovies = np.random.uniform(-0.01, 0.01, size=(N, totalMovies)).astype(np.float32)


A_gpu_matrixAUsers = cuda.mem_alloc(matrixAUsers.nbytes)
B_gpu_matrixBMovies = cuda.mem_alloc(matrixBMovies.nbytes)


if (B_gpu_matrixBMovies==None or A_gpu_matrixAUsers==None):
    print("Damn")


rows = (Row * len(rows_data))(*rows_data)

# Allocate memory on the GPU for the rows
rows_gpu = cuda.mem_alloc(ctypes.sizeof(rows))
if (rows_gpu==None):
    print("damn")

# Transfer the rows data to the GPU
cuda.memcpy_htod(rows_gpu, rows)



cuda.memcpy_htod(A_gpu_matrixAUsers, matrixAUsers)
cuda.memcpy_htod(B_gpu_matrixBMovies, matrixBMovies)




matmul = mod.get_function("matmul")

# Define grid and block sizes
block_size = (16, 16, 1)  # 16x16 threads per block
grid_size = ((totalRaters + block_size[1] - 1) // block_size[1],(totalMovies + block_size[0] - 1) // block_size[0])
print(f"Matrix A shape: {matrixAUsers.shape}")
print(f"Matrix B shape: {matrixBMovies.shape}")



# Run the parallel CUDA kernel
cuda.Context.synchronize()  # Ensure synchronization before execution
start_time = time.time()
matmul(A_gpu_matrixAUsers,B_gpu_matrixBMovies ,np.int32(N), np.int32(totalRaters),np.int32(totalMovies),rows_gpu,block=block_size, grid=grid_size)
cuda.Context.synchronize()  # Ensure synchronization after execution

# Copy result back to CPU

# Track execution time for CUDA
end_time = time.time()
print(f"Parallel GPU Execution Time (CUDA): {end_time - start_time:.5f} seconds")

Matrix A shape: (200948, 50)
Matrix B shape: (50, 292757)


KeyboardInterrupt: 